In [8]:
#!pip install lightgbm
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [9]:
import pandas as pd

In [41]:
train = pd.read_csv('Additional_train.csv')
test = pd.read_csv('Additional_test.csv')

In [42]:
# binning for hard inquiries
# AnnualSalary * TotalBalance
# AnnualSalary * CreditScore
# CreditScore * CreditScore
# Age * CreditScore
# Age * AnnualSalary
# Age * TotalBalance
train['SalaryBalance'] = train['AnnualSalary'] * train['TotalBalance']
train['SalaryCredit'] = train['AnnualSalary'] * train['CreditScore']
train['CreditSquared'] = train['CreditScore'] * train['CreditScore']
train['AgeCredit'] = train['Age'] * train['CreditScore']
train['AgeSalary'] = train['Age'] * train['AnnualSalary']
train['AgeBalance'] = train['Age'] * train['TotalBalance']

test['SalaryBalance'] = test['AnnualSalary'] * test['TotalBalance']
test['SalaryCredit'] = test['AnnualSalary'] * test['CreditScore']
test['CreditSquared'] = test['CreditScore'] * test['CreditScore']
test['AgeCredit'] = test['Age'] * test['CreditScore']
test['AgeSalary'] = test['Age'] * test['AnnualSalary']
test['AgeBalance'] = test['Age'] * test['TotalBalance']

train.head()

# Previous HardInquiries
train['PrevInquiries'] = train.groupby('CustomerID')['HardInquiries'].cumsum()
test['PrevInquiries'] = test.groupby('CustomerID')['HardInquiries'].cumsum()

train = train.set_index(['CustomerID', 'Week'])
shifted = train.groupby('CustomerID').shift(periods=1)
test = test.set_index(['CustomerID', 'Week'])
shifted_test = test.groupby('CustomerID').shift(periods=1)
# Lagged Utilisation
train = train.join(shifted['Utilisation'].rename('LaggedUtilisation'))
test = test.join(shifted_test['Utilisation'].rename('LaggedUtilisation'))

# Lagged PaymentRatio
train = train.join(shifted['PaymentRatio'].rename('LaggedPaymentRatio'))
test = test.join(shifted_test['PaymentRatio'].rename('LaggedPaymentRatio'))

# Rolling5WeekUtilisation
train['Rolling5WeekUtil'] = train['Utilisation'].rolling(window=5).mean()
test['Rolling5WeekUtil'] = test['Utilisation'].rolling(window=5).mean()

# Rolling5WeekPaymentRatio
train['Rolling5WeekPaymentRatio'] = train['PaymentRatio'].rolling(window=5).mean()
test['Rolling5WeekPaymentRatio'] = test['PaymentRatio'].rolling(window=5).mean()

train.head()

train.to_csv('interactions_train.csv')
test.to_csv('interactions_test.csv')